In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
import time
from sqlalchemy import inspect, create_engine
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

# Create brower location to pull 

In [5]:
#Website searching Data Analyst positions
indeed = "https://www.indeed.com/jobs?q=data+analyst"

browser.visit(indeed)



In [6]:
html = browser.html
html[0:500]

'<html dir="ltr" lang="en" class="js-focus-visible" data-js-focus-visible=""><head>\n    <link rel="shortcut icon" href="/images/favicon.ico">\n    <title>Data Analyst Jobs, Employment | Indeed.com</title>\n    <meta http-equiv="content-type" content="text/html; charset=utf-8">\n    <meta name="description" content="28,052 Data Analyst jobs available on Indeed.com. Apply to Data Analyst, Entry Level Data Analyst, Senior Data Analyst and more!">\n    <meta name="keywords" content="Data Analyst Jobs, Em'

In [8]:
soup= BeautifulSoup(browser.html, 'lxml')
type(soup)

bs4.BeautifulSoup

In [9]:
#Find code to loop through web page and pull the title of the job and possibly the location, then find code to click the job and acquire the salary.
jobs = soup.find_all('div', {'class': 'cardOutline'})
len(jobs)


15

In [11]:
#Create for loops 
rows = []
browser.visit(indeed)
for i in range(1, 50):
    
    html = browser.html
    soup = BeautifulSoup(browser.html, "html.parser")
    jobs = soup.find_all('div', {'class': 'cardOutline'})
    
    
    for job in jobs:
        title = job.find('a', {"class": 'jcs-JobTitle'}).text
        link = "www.indeed.com" + job.find('a', {"class": 'jcs-JobTitle'})["href"]

        company_name = job.find('span', {'class': 'companyName'})
        if company_name: 
            company = company_name.text
        else:
            company = None
        
        company_con = job.find('a', {"class": 'companyOverviewLink'})
        if company_con: 
            company_link = "www.indeed.com" + company_con["href"]
        else:
            company_link = None

        rating_con = job.find('span', {"class": 'ratingNumber'})
        if rating_con: 
            rating = rating_con.text
        else:
            rating = None


        loc = job.find('div', {"class": 'companyLocation'}).text

        salary_con = job.find('div', {"class": 'salary-snippet-container'})
        if salary_con: 
            salary = salary_con.text
        else:
            salary = None


        status_con = job.find('svg', {"aria-label": "Job type"})
        if status_con:
            status = status_con.parent.text
        else:
            status = None

        data = {}
        data["job_title"] = title
        data["job_link"] = link
        data["company"] = company
        data["company_link"] = company_link
        data["company_rating"] = rating
        data["location"] = loc
        data["salary"] = salary
        data["status"] = status
        rows.append(data)
    
    browser.links.find_by_text(f"{i+1}").click()
        
df = pd.DataFrame(rows)
df.head()

,job_title,job_link,company,company_link,company_rating,location,salary,status
0,Behavior Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CX...,Centre For Neuro Skills,www.indeed.com/cmp/Centre-For-Neuro-Skills,3.4,"Bakersfield, CA 93313 (Silver Creek area)",None,Full-time
1,Sr. Business Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Cz...,MidFirst Bank,www.indeed.com/cmp/Midfirst-Bank,3.4,"Oklahoma City, OK 73118",None,Full-time
2,"Data, Reporting, and CRM Specialist",www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0D6...,"BrandTech Scientific, Inc.",None,None,"Hybrid remote in Essex, CT 06426","$65,000 - $75,000 a year Employer est.",Full-time
3,Board Certified Behavior Analyst (BCBA),www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BP...,Pathways Autism Center,www.indeed.com/cmp/Pathways-Behavioral-Consult...,2.5,"Atlanta, GA 30338","$70,000 - $90,000 a year Employer est.",Full-time
4,Business Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0C_...,Piper Aircraft Inc,www.indeed.com/cmp/Piper-Aircraft-Inc,3.2,"Vero Beach, FL 32960 (Vero Beach Regional Airp...",None,Full-time


In [12]:
len(df)

735

In [13]:
df["company_rating"] = pd.to_numeric(df.company_rating)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   job_title       735 non-null    object 
 1   job_link        735 non-null    object 
 2   company         733 non-null    object 
 3   company_link    530 non-null    object 
 4   company_rating  530 non-null    float64
 5   location        735 non-null    object 
 6   salary          355 non-null    object 
 7   status          555 non-null    object 
dtypes: float64(1), object(7)
memory usage: 46.1+ KB


In [14]:
#combine with regex s=space d=catch or match all numbers \+ = plus sign matches plus sign, regex meaning.
df["status"] = df.status.str.replace("\s\+\d","")

C:\Users\19722\AppData\Local\Temp\ipykernel_15192\152645899.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["status"] = df.status.str.replace("\s\+\d","")


In [15]:
df.status.value_counts()

Full-time     440
Contract      104
Part-time       7
Temporary       2
Internship      1
Seasonal        1
Name: status, dtype: int64

In [16]:
#Status column -  ID, name, last_updated
df.status.unique()

array(['Full-time', 'Contract', None, 'Internship', 'Part-time',
       'Temporary', 'Seasonal'], dtype=object)

In [17]:
#Status lookup table
status = df.status.dropna().unique()
status_df = pd.DataFrame()
status_df["status"] = status
status_df

,status
0,Full-time
1,Contract
2,Internship
3,Part-time
4,Temporary
5,Seasonal


In [18]:
#Company lookup table, revised to drop the 2nd Insight Global below
companies = df.loc[:,["company","company_link", "company_rating"]].drop_duplicates().reset_index(drop=True)
companies.company.value_counts()

Centre For Neuro Skills                      1
MidFirst Bank                                1
Disney Media & Entertainment Distribution    1
DoorDash, Inc.                               1
Pearson                                      1
                                            ..
Granify                                      1
Allied Stone Inc                             1
Live Nation                                  1
X2Logics                                     1
SCO Family of Services                       1
Name: company, Length: 576, dtype: int64

In [19]:
companies.loc[companies.company == "Insight Global"]

,company,company_link,company_rating
128,Insight Global,www.indeed.com/cmp/Insight-Global,3.9


In [20]:
df.loc[df.company == "Insight Global", "company_link"] = "www.indeed.com/cmp/Insight-Global"
df.loc[df.company == "Insight Global", "company_rating"] = 3.9

In [21]:
companies = df.loc[:,["company","company_link", "company_rating"]].drop_duplicates().reset_index(drop=True)
companies.company.value_counts()

Centre For Neuro Skills                      1
MidFirst Bank                                1
Disney Media & Entertainment Distribution    1
DoorDash, Inc.                               1
Pearson                                      1
                                            ..
Granify                                      1
Allied Stone Inc                             1
Live Nation                                  1
X2Logics                                     1
SCO Family of Services                       1
Name: company, Length: 576, dtype: int64

In [22]:
#Companies lookup table
companies

,company,company_link,company_rating
0,Centre For Neuro Skills,www.indeed.com/cmp/Centre-For-Neuro-Skills,3.4
1,MidFirst Bank,www.indeed.com/cmp/Midfirst-Bank,3.4
2,"BrandTech Scientific, Inc.",None,NaN
3,Pathways Autism Center,www.indeed.com/cmp/Pathways-Behavioral-Consult...,2.5
4,Piper Aircraft Inc,www.indeed.com/cmp/Piper-Aircraft-Inc,3.2
...,...,...,...
572,ARCXIS,www.indeed.com/cmp/Arcxis,3.1
573,LeewayHertz,None,NaN
574,Citizens,www.indeed.com/cmp/Citizens-2,3.4
575,New York University,www.indeed.com/cmp/New-York-University,4.2


In [23]:
#State 
def getState(loc):
    if "," in loc:
        state = loc.replace("\xa0", " ").split(",")[1].strip().split(" ")[0].strip()
    else:
        state = None
        
    return(state)

In [24]:
#Find the locations with"Downtown area"
getState("Austin, TX (Downtown area)")

'TX'

In [25]:
#
df["state"] = df.location.apply(lambda x: getState(x))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   job_title       735 non-null    object 
 1   job_link        735 non-null    object 
 2   company         733 non-null    object 
 3   company_link    530 non-null    object 
 4   company_rating  530 non-null    float64
 5   location        735 non-null    object 
 6   salary          355 non-null    object 
 7   status          555 non-null    object 
 8   state           545 non-null    object 
dtypes: float64(1), object(8)
memory usage: 51.8+ KB


In [26]:
#
df.state.value_counts()

TX    218
CA     73
NY     48
VA     21
FL     15
GA     15
IL     15
WA     13
MN     12
PA     10
CO      9
NC      9
MA      8
OK      7
AZ      7
NJ      6
MS      5
TN      5
MI      5
CT      4
DC      4
UT      4
MO      4
IA      4
WI      3
OR      3
DE      2
MD      2
LA      2
SC      2
OH      2
VT      1
NV      1
NE      1
NH      1
KS      1
KY      1
AL      1
NM      1
Name: state, dtype: int64

State lookup table

In [29]:
#State look-up table
state = df.state.dropna().unique()
state_df = pd.DataFrame()
state_df["state"] = state
state_df

,state
0,CA
1,OK
2,CT
3,GA
4,FL
5,TX
6,AL
7,NY
8,OR
9,WI


In [30]:
#
def getRemote(loc):
    if "remote" in loc.lower():
        return(True)
    else:
        return(False)
        
  

In [31]:
#
df["is_remote"] = df.location.apply(lambda x: getRemote(x))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   job_title       735 non-null    object 
 1   job_link        735 non-null    object 
 2   company         733 non-null    object 
 3   company_link    530 non-null    object 
 4   company_rating  530 non-null    float64
 5   location        735 non-null    object 
 6   salary          355 non-null    object 
 7   status          555 non-null    object 
 8   state           545 non-null    object 
 9   is_remote       735 non-null    bool   
dtypes: bool(1), float64(1), object(8)
memory usage: 52.5+ KB


In [32]:
df.is_remote.value_counts()

True     368
False    367
Name: is_remote, dtype: int64

In [33]:
df.is_remote.info()

<class 'pandas.core.series.Series'>
RangeIndex: 735 entries, 0 to 734
Series name: is_remote
Non-Null Count  Dtype
--------------  -----
735 non-null    bool 
dtypes: bool(1)
memory usage: 863.0 bytes


In [34]:
is_remote = df.is_remote.dropna().unique()
is_remote_df = pd.DataFrame()
is_remote_df["is_remote"] = is_remote
is_remote_df

,is_remote
0,False
1,True


In [35]:
#Find the companies paying for ads
def getAd(link):
    if "pagead" in link.lower():
        return(True)
    else:
        return(False)

In [36]:
df["is_ad"] = df.job_link.apply(lambda x: getAd(x))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   job_title       735 non-null    object 
 1   job_link        735 non-null    object 
 2   company         733 non-null    object 
 3   company_link    530 non-null    object 
 4   company_rating  530 non-null    float64
 5   location        735 non-null    object 
 6   salary          355 non-null    object 
 7   status          555 non-null    object 
 8   state           545 non-null    object 
 9   is_remote       735 non-null    bool   
 10  is_ad           735 non-null    bool   
dtypes: bool(2), float64(1), object(8)
memory usage: 53.2+ KB


In [37]:
df.is_ad.value_counts()

False    611
True     124
Name: is_ad, dtype: int64

In [38]:
#Paying for Ads, add to main job table as bool
df.groupby("company").is_ad.sum().sort_values(ascending=False)

company
Insight Global                             5
DISYS                                      4
USAA                                       4
Signify Health                             3
Vaco                                       3
                                          ..
General Motors                             0
General Dynamics Information Technology    0
GeneDx                                     0
Gainwell Technologies LLC                  0
pulseData                                  0
Name: is_ad, Length: 576, dtype: int64

In [39]:
#Create a CSV for companies 
#companies.to_csv('companies.csv', index=False)

In [40]:
#companies_file = pd.read_csv('companies.csv')
#companies_file

In [41]:
#Create a CSV for States
#state_df.to_csv('state.csv', index=False)

In [42]:
#state_file = pd.read_csv('state.csv')
#state_file

In [43]:
#Create a CSV for status (Full-time/part-time)
#status_df.to_csv('status.csv', index=False)

In [44]:
#status_file = pd.read_csv('status.csv')
#status_file

In [45]:
import pandas as pd
from sqlalchemy import create_engine, inspect

In [46]:
USERNAME = "postgres"
PASSWORD = "fAr3st!290"
URL = "localhost"
PORT = 5432
DATABASE = "Jobs_db"

connection_str = f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{URL}:{PORT}/{DATABASE}"
#print(connection_str)

engine = create_engine(connection_str)

In [47]:
# create the inspector and connect to the engine
inspector_gadget = inspect(engine)

tables = inspector_gadget.get_table_names()
for table in tables:
    print(table)
    
    # get all columns in table
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column)
    print()

status
{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': "nextval('status_id_seq'::regclass)", 'autoincrement': True, 'comment': None}
{'name': 'status', 'type': VARCHAR(length=20), 'nullable': False, 'default': None, 'autoincrement': False, 'comment': None}
{'name': 'last_updated', 'type': TIMESTAMP(), 'nullable': False, 'default': 'LOCALTIMESTAMP', 'autoincrement': False, 'comment': None}

state
{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': "nextval('state_id_seq'::regclass)", 'autoincrement': True, 'comment': None}
{'name': 'state', 'type': VARCHAR(length=10), 'nullable': False, 'default': None, 'autoincrement': False, 'comment': None}
{'name': 'last_updated', 'type': TIMESTAMP(), 'nullable': False, 'default': 'LOCALTIMESTAMP', 'autoincrement': False, 'comment': None}

company
{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': "nextval('company_id_seq'::regclass)", 'autoincrement': True, 'comment': None}
{'name': 'company', 'type': VARCHA

In [48]:
status_df.head()

,status
0,Full-time
1,Contract
2,Internship
3,Part-time
4,Temporary


In [49]:
engine.execute("truncate table status restart identity cascade")
status_df.to_sql("status", engine, method="multi", if_exists = "append", index=False)

6

In [50]:
engine.execute("truncate table state restart identity cascade")
state_df.to_sql("state", engine, method="multi", if_exists = "append", index=False)

39

In [51]:
#companies.to_sql("company", engine, method="multi", if_exists = "append", index=False)

In [52]:
#companies.head()

In [53]:
#companies.info()

In [54]:
#companies.loc[pd.isnull(companies.company)]

In [55]:
#Drop one company with no name, inclue in writeup
companies = companies.loc[pd.notnull(companies.company)].reset_index(drop=True)
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   company         576 non-null    object 
 1   company_link    392 non-null    object 
 2   company_rating  392 non-null    float64
dtypes: float64(1), object(2)
memory usage: 13.6+ KB


In [56]:
engine.execute("truncate table company restart identity cascade")
companies.to_sql("company", engine, method="multi", if_exists = "append", index=False)

576

In [57]:
df.head()

,job_title,job_link,company,company_link,company_rating,location,salary,status,state,is_remote,is_ad
0,Behavior Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CX...,Centre For Neuro Skills,www.indeed.com/cmp/Centre-For-Neuro-Skills,3.4,"Bakersfield, CA 93313 (Silver Creek area)",None,Full-time,CA,False,True
1,Sr. Business Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Cz...,MidFirst Bank,www.indeed.com/cmp/Midfirst-Bank,3.4,"Oklahoma City, OK 73118",None,Full-time,OK,False,True
2,"Data, Reporting, and CRM Specialist",www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0D6...,"BrandTech Scientific, Inc.",None,NaN,"Hybrid remote in Essex, CT 06426","$65,000 - $75,000 a year Employer est.",Full-time,CT,True,True
3,Board Certified Behavior Analyst (BCBA),www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BP...,Pathways Autism Center,www.indeed.com/cmp/Pathways-Behavioral-Consult...,2.5,"Atlanta, GA 30338","$70,000 - $90,000 a year Employer est.",Full-time,GA,False,True
4,Business Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0C_...,Piper Aircraft Inc,www.indeed.com/cmp/Piper-Aircraft-Inc,3.2,"Vero Beach, FL 32960 (Vero Beach Regional Airp...",None,Full-time,FL,False,True


In [58]:
status2 = pd.read_sql("select id as status_id, status from status", engine)
status2.head(20)

,status_id,status
0,1,Full-time
1,2,Contract
2,3,Internship
3,4,Part-time
4,5,Temporary
5,6,Seasonal


In [59]:
df = pd.merge(df, status2, on = "status", how ="left")
df.drop("status", axis=1, inplace=True)
df.head()

,job_title,job_link,company,company_link,company_rating,location,salary,state,is_remote,is_ad,status_id
0,Behavior Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CX...,Centre For Neuro Skills,www.indeed.com/cmp/Centre-For-Neuro-Skills,3.4,"Bakersfield, CA 93313 (Silver Creek area)",None,CA,False,True,1.0
1,Sr. Business Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Cz...,MidFirst Bank,www.indeed.com/cmp/Midfirst-Bank,3.4,"Oklahoma City, OK 73118",None,OK,False,True,1.0
2,"Data, Reporting, and CRM Specialist",www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0D6...,"BrandTech Scientific, Inc.",None,NaN,"Hybrid remote in Essex, CT 06426","$65,000 - $75,000 a year Employer est.",CT,True,True,1.0
3,Board Certified Behavior Analyst (BCBA),www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BP...,Pathways Autism Center,www.indeed.com/cmp/Pathways-Behavioral-Consult...,2.5,"Atlanta, GA 30338","$70,000 - $90,000 a year Employer est.",GA,False,True,1.0
4,Business Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0C_...,Piper Aircraft Inc,www.indeed.com/cmp/Piper-Aircraft-Inc,3.2,"Vero Beach, FL 32960 (Vero Beach Regional Airp...",None,FL,False,True,1.0


In [60]:
df.head()

,job_title,job_link,company,company_link,company_rating,location,salary,state,is_remote,is_ad,status_id
0,Behavior Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CX...,Centre For Neuro Skills,www.indeed.com/cmp/Centre-For-Neuro-Skills,3.4,"Bakersfield, CA 93313 (Silver Creek area)",None,CA,False,True,1.0
1,Sr. Business Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Cz...,MidFirst Bank,www.indeed.com/cmp/Midfirst-Bank,3.4,"Oklahoma City, OK 73118",None,OK,False,True,1.0
2,"Data, Reporting, and CRM Specialist",www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0D6...,"BrandTech Scientific, Inc.",None,NaN,"Hybrid remote in Essex, CT 06426","$65,000 - $75,000 a year Employer est.",CT,True,True,1.0
3,Board Certified Behavior Analyst (BCBA),www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BP...,Pathways Autism Center,www.indeed.com/cmp/Pathways-Behavioral-Consult...,2.5,"Atlanta, GA 30338","$70,000 - $90,000 a year Employer est.",GA,False,True,1.0
4,Business Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0C_...,Piper Aircraft Inc,www.indeed.com/cmp/Piper-Aircraft-Inc,3.2,"Vero Beach, FL 32960 (Vero Beach Regional Airp...",None,FL,False,True,1.0


In [61]:
state2 = pd.read_sql("select id as state_id, state from state", engine)
df = pd.merge(df, state2, on = "state", how ="left")
df.drop("state", axis=1, inplace=True)
df.head()

,job_title,job_link,company,company_link,company_rating,location,salary,is_remote,is_ad,status_id,state_id
0,Behavior Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CX...,Centre For Neuro Skills,www.indeed.com/cmp/Centre-For-Neuro-Skills,3.4,"Bakersfield, CA 93313 (Silver Creek area)",None,False,True,1.0,1.0
1,Sr. Business Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Cz...,MidFirst Bank,www.indeed.com/cmp/Midfirst-Bank,3.4,"Oklahoma City, OK 73118",None,False,True,1.0,2.0
2,"Data, Reporting, and CRM Specialist",www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0D6...,"BrandTech Scientific, Inc.",None,NaN,"Hybrid remote in Essex, CT 06426","$65,000 - $75,000 a year Employer est.",True,True,1.0,3.0
3,Board Certified Behavior Analyst (BCBA),www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BP...,Pathways Autism Center,www.indeed.com/cmp/Pathways-Behavioral-Consult...,2.5,"Atlanta, GA 30338","$70,000 - $90,000 a year Employer est.",False,True,1.0,4.0
4,Business Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0C_...,Piper Aircraft Inc,www.indeed.com/cmp/Piper-Aircraft-Inc,3.2,"Vero Beach, FL 32960 (Vero Beach Regional Airp...",None,False,True,1.0,5.0


In [62]:
company2 = pd.read_sql("select id as company_id, company from company", engine)
df = pd.merge(df, company2, on = "company", how ="left")
df.drop(["company", "company_link", "company_rating"], axis=1, inplace=True)
df.head()

,job_title,job_link,location,salary,is_remote,is_ad,status_id,state_id,company_id
0,Behavior Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CX...,"Bakersfield, CA 93313 (Silver Creek area)",None,False,True,1.0,1.0,1.0
1,Sr. Business Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Cz...,"Oklahoma City, OK 73118",None,False,True,1.0,2.0,2.0
2,"Data, Reporting, and CRM Specialist",www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0D6...,"Hybrid remote in Essex, CT 06426","$65,000 - $75,000 a year Employer est.",True,True,1.0,3.0,3.0
3,Board Certified Behavior Analyst (BCBA),www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BP...,"Atlanta, GA 30338","$70,000 - $90,000 a year Employer est.",False,True,1.0,4.0,4.0
4,Business Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0C_...,"Vero Beach, FL 32960 (Vero Beach Regional Airp...",None,False,True,1.0,5.0,5.0


In [63]:
#Database
df.to_sql("job", engine, method="multi", if_exists = "append", index=False)

735

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 735 entries, 0 to 734
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   job_title   735 non-null    object 
 1   job_link    735 non-null    object 
 2   location    735 non-null    object 
 3   salary      355 non-null    object 
 4   is_remote   735 non-null    bool   
 5   is_ad       735 non-null    bool   
 6   status_id   555 non-null    float64
 7   state_id    545 non-null    float64
 8   company_id  733 non-null    float64
dtypes: bool(2), float64(3), object(4)
memory usage: 47.4+ KB


In [65]:
#Don't worry about it
# def getCity(loc):
#     if "," in loc:
#         city_list = loc.replace("\xa0", " ").split(",")[0].strip().split(" ")
#         city = city_list[0] if len(city_list) ==1 else city_list[1]
#     else:
#         city = None
        
#     return(city)

In [ ]:
# df["city"] = df.location.apply(lambda x: getCity(x))
# df.info()

In [ ]:
# df.city.value_counts()

In [ ]:
# df.loc[df.location.str.contains("in")]

In [ ]:
#Set-up sleep function if needed
# browser.visit(dice_lyst)
# time.sleep(2)